In [1]:
!pip install modin
!pip install ray
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 5.0 MB/s 
     |████████████████████████████████| 140 kB 44.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 123 kB/s 
     |████████████████████████████████| 4.1 MB 55.3 MB/s 
     |████████████████████████████████| 10.1 MB 39.8 MB/s 
     |████████████████████████████████| 144 kB 66.1 MB/s 
     |████████████████████████████████| 461 kB 51.3 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.46.3
    Uninstalling grpcio-1.46.3:
      Successfully uninstalled grpcio-1.46.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Read CSV with Pandas, Dask, Modin/Ray**

In [2]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import time

In [3]:
csvfile='C:\Users\jeeye\Documents\DG\price_paid_records.csv'

SyntaxError: ignored

In [ ]:
#Pandas to read the CSV

start = time.time()
df=pd.read_csv(csvfile)
end = time.time()
print('Time to read the CSV (pandas): ',end - start, 'seconds')

In [ ]:
#Modin/Ray to read the CSV

start = time.time()
ray.shutdown()
ray.init()
mpd.read_csv(csvfile)
end = time.time()
print('Time to read the CSV (modin/ray): ',end - start, 'seconds')

In [ ]:
#Dask to read the CSV

start = time.time()
dd.read_csv(csvfile)
end = time.time()
print('Time to read the CSV (dask): ',end - start, 'seconds')

**Dask took the least time reading the CSV file!**


# **Clean the Column Names**

In [ ]:
# Remove spaces and special chars from the cols

data.columns=data.columns.str.lower()
data.columns = data.columns.str.replace(' ', '')
data.columns=data.columns.str.replace('[^\w]','_',regex=True)
print(data.columns)

# **Validation**



In [ ]:
%%writefile utility.py
import yaml
import logging
import os
import subprocess
import pandas as pd
import re
import gc
import datetime

def read_config_file(filepath):
  with open(filepath, 'r') as stream:
    try:
      return yaml.safe_load(stream)
    except yaml.YAMLerror as exc:
      logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df, table_config):
  df.columns=df.columns.str.lower()
  df.columns=df.columns.str.replace('[^\w]','_',regex=True)
  df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
  df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
  expected_col = list(map(lambda x: x.lower(), table_config['columns']))
  expected_col.sort()
  df.columns = list(map(lambda x: x.lower(), list(df.columns)))
  df=df.reindex(sorted(df.columns), axis=1)
  if len(df.columns) == len(expected_col) and list(expected_col) == list(df.columns):
    print("Column name and column length validation passed")
    return 1
  else:
    print("Column name and column length validation failed")
    mismatched_columns_file = list(set(df.columns).difference(expected_col))
    print("Following file columns are not in the YAML file", mismatched_columns_file)
    missing_YAML_file = list(set(expected_col).difference(df.columns))
    print("Following YAML columns are not in the file uploaded", missing_YAML_file)
    logging.info(f'df columns: {df.columns}')
    logging.info(f'expected columns: {expected_col}')
    return 0

In [ ]:
%%writefile store.yaml
file_type: csv
dataset_name: testfile
file_name: price_paid_records
table_name: endsurv
inbound_delimiter: ','
outbound_delimiter: '|'
skip_leading_rows: 1
columns:
  - transaction_unique_identifier
  - price
  - date_of_transfer
  - property_type
  - old_new
  - duration
  - town_city
  - district
  - county
  - ppdcategory_type
  - record_status_monthly_file_only

In [ ]:
# Read config file

import utility as util
config_data = util.read_config_file("store.yaml")

In [ ]:
config_data['inbound_delimiter']

In [ ]:
#data of the config file
config_data

In [ ]:
#Read the file using config file
file_type = config_data['file_type']
source_file = "C:\Users\jeeye\Documents\DG\" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

In [ ]:
#Validate the Header of the file
util.col_header_val(df,config_data)

In [ ]:
print("Columns of files are:" , df.columns)
print("Columns of YAML are:" , config_data['columns'])

In [ ]:
if util.col_header_val(df,config_data)==0:
    print("Validation failed")
    print("Columns of the file does not match the YAML")
else:
    print("Column validation passed")
    print('Preview of the data\n', df.head())

# **Save pipe separated file as .gz**

In [ ]:
#csv to gz

import gzip
import csv

df.to_csv('dfgz.csv.gz',
      sep='|',
      header=True,
      index=False,
      quoting=csv.QUOTE_ALL,
      compression='gzip',
      quotechar='"',
      doublequote=True,
      line_terminator='\n')

In [ ]:
# Number of cols in the gz file
import os
entries = os.listdir('./dfgz.csv.gz')
for entry in entries:
    print(entry)

# **Summary of the File**

In [ ]:
#Size of the CSV file

import os
data=dd.read_csv(csvfile)
print('Size of the CSV file: ', str(os.path.getsize(csvfile), 'Bytes'))

In [ ]:
#Number of rows/cols
print('Number of Rows: ',len(df.rows))
print('Number of columns:', len(df.columns))